# Подготовка данных

**D_work**

Описание статусов относительно работы:
- ID — идентификатор социального статуса клиента относительно работы;
- COMMENT — расшифровка статуса.


**D_pens**

Описание статусов относительно пенсии:
- ID — идентификатор социального статуса;
- COMMENT — расшифровка статуса.


**D_clients**

Описание данных клиентов:
- ID — идентификатор записи;
- AGE	— возраст клиента;
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- EDUCATION — образование;
- MARITAL_STATUS — семейное положение;
- CHILD_TOTAL	— количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- SOCSTATUS_WORK_FL	— социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL	— социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- REG_ADDRESS_PROVINCE — область регистрации клиента;
- FACT_ADDRESS_PROVINCE — область фактического пребывания клиента;
- POSTAL_ADDRESS_PROVINCE — почтовый адрес области;
- FL_PRESENCE_FL — наличие в собственности квартиры (1 — есть, 0 — нет);
- OWN_AUTO — количество автомобилей в собственности.


**D_agreement**

Таблица с зафиксированными откликами клиентов на предложения банка:
- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- ID_CLIENT — идентификатор клиента;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было).
    
    
**D_job**

Описание информации о работе клиентов:
- GEN_INDUSTRY — отрасль работы клиента;
- GEN_TITLE — должность;
- JOB_DIR — направление деятельности внутри компании;
- WORK_TIME — время работы на текущем месте (в месяцах);
- ID_CLIENT — идентификатор клиента.


**D_salary**

Описание информации о заработной плате клиентов:
- ID_CLIENT — идентификатор клиента;
- FAMILY_INCOME — семейный доход (несколько категорий);
- PERSONAL_INCOME — личный доход клиента (в рублях).


**D_last_credit**

Информация о последнем займе клиента:
- ID_CLIENT — идентификатор клиента;
- CREDIT — сумма последнего кредита клиента (в рублях);
- TERM — срок кредита;
- FST_PAYMENT — первоначальный взнос (в рублях).


**D_loan**

Информация о кредитной истории клиента:
- ID_CLIENT — идентификатор клиента;
- ID_LOAN — идентификатор кредита.

**D_close_loan**

Информация о статусах кредита (ссуд):
- ID_LOAN — идентификатор кредита;
- CLOSED_FL — текущий статус кредита (1 — закрыт, 0 — не закрыт).

Ниже представлен минимальный список колонок, которые должны находиться в итоговом датасете после склейки и агрегации данных. По своему усмотрению вы можете добавить дополнительные к этим колонки.

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


Будьте внимательны при сборке датасета: это реальные банковские данные, в которых могут наблюдаться дубли, некорректно заполненные значения или значения, противоречащие друг другу. Для получения качественной модели необходимо предварительно очистить датасет от такой информации.

In [18]:
import logging
import os

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import StratifiedKFold, cross_val_score

pd.set_option("display.max_columns", 50)

logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)


def drop_full_duplicates(table: pd.DataFrame) -> pd.DataFrame:
    """Check for full duplicates in table, log info."""
    table_with_dropped_dupluicate = table.drop_duplicates()
    shape = table.shape
    shape_with_dropped_duplicate = table_with_dropped_dupluicate.shape
    if shape != shape_with_dropped_duplicate:
        LOGGER.warning(
            "Found full duplicates, shape without drop duplicate - %s, with drop - %s.",
            shape,
            shape_with_dropped_duplicate,
        )
    return table_with_dropped_dupluicate

In [19]:
# Константы

DATA_PATH = "../data/" # путь до директории с CSV таблицами
MAIN_TABLE_NAME = "TARGET" # название основной таблицы (остальные будет присоединяться к ней справа)

RESULT_TABLE_NAME = "PREPARED_DATA"

In [20]:
# 1. Читаем все таблицы из CSV файлов
# 2. Проверяем на полные дубли и удаляем, если такие есть
# 3. Сохраняем таблицы в словарь

tables_filenames = os.listdir(DATA_PATH)
tables = {}
for table_filename in tables_filenames:
    if table_filename == f"{RESULT_TABLE_NAME}.csv":
        continue
    
    _, table_name = table_filename.split("_", maxsplit=1)
    table_name, _fmt = table_name.split(".")

    if table_name.endswith("s"):
        table_name = table_name[:-1]

    final_path = os.path.join(DATA_PATH, table_filename)
    LOGGER.info("Reading table from %s.", final_path)
    table = pd.read_csv(final_path)
    table_name = table_name.upper()
    if "ID" in table.columns:
        table = table.rename(columns={"ID": f"ID_{table_name}"})
    
    tables[table_name] = drop_full_duplicates(table=table)
    LOGGER.info("Table saved with name - %s.", table_name)

INFO:__main__:Reading table from ../data/D_clients.csv.
INFO:__main__:Table saved with name - CLIENT.
INFO:__main__:Reading table from ../data/D_close_loan.csv.
INFO:__main__:Table saved with name - CLOSE_LOAN.
INFO:__main__:Reading table from ../data/D_target.csv.
INFO:__main__:Table saved with name - TARGET.
INFO:__main__:Reading table from ../data/D_work.csv.
INFO:__main__:Table saved with name - WORK.
INFO:__main__:Reading table from ../data/D_loan.csv.
INFO:__main__:Table saved with name - LOAN.
INFO:__main__:Reading table from ../data/D_pens.csv.
INFO:__main__:Table saved with name - PEN.
INFO:__main__:Reading table from ../data/D_job.csv.
INFO:__main__:Table saved with name - JOB.
INFO:__main__:Reading table from ../data/D_last_credit.csv.
INFO:__main__:Table saved with name - LAST_CREDIT.
INFO:__main__:Reading table from ../data/D_salary.csv.
INFO:__main__:Table saved with name - SALARY.


In [21]:
# 1. Берём исходную таблицу с TARGET в качестве основной, поскольу по условию AGREEMENT_RK из этой таблицы идентифицирует объект.
# 2. Проходимся по всем остальным таблицам и пробуем сделать LEFT JOIN к исходной таблице
# 3. Проверяем, что нет полных дублей
# 4. Заполняем пропуски, если у клиента нет работы
# 5. Агрегируем, чтобы получить - LOAN_NUM_TOTAL и LOAN_NUM_CLOSED
# 5. Сохраняем итоговую таблицу

main_table = tables.pop(MAIN_TABLE_NAME)
while set(tables.keys()) != set(["WORK", "PEN"]): # WORK/PEN просто содержат расшифрофку числовых значений и не несут доп. информации
    current_keys = list(filter(lambda x: x.startswith("ID_"), main_table.columns))
    for table_name, table in list(tables.items()):
        join_keys = list(set(table.columns) & set(current_keys))
        if len(join_keys) == 1:
            table = table = tables.pop(table_name)
            break
        elif len(join_keys) == 0:
            continue
        else:
            raise NotImplementedError(f"Found too many join keys, {join_keys}.")
    else:
        continue

    LOGGER.info("Left join current table with '%s' table on - %s.", table_name, join_keys)
    main_table = main_table.merge(table, on=join_keys, how="left")
    LOGGER.info("Output shape is %s.", main_table.shape)

LOGGER.info("Result table shape before drop duplicates and aggregation is %s.", main_table.shape)
main_table = drop_full_duplicates(table=main_table)
main_table.fillna(
    {
        "GEN_INDUSTRY": "БЕЗ РАБОТЫ",
        "GEN_TITLE": "БЕЗ РАБОТЫ",
        "JOB_DIR": "БЕЗ РАБОТЫ",
        "WORK_TIME": 0,
    },
    inplace=True,
)

group_key = list(set(main_table.columns) - {"ID_LOAN", "CLOSED_FL", })
LOGGER.info("Aggregate data with key %s.", group_key)
load_num_total_func = pd.NamedAgg("ID_LOAN", "count")
load_num_closed_func = pd.NamedAgg("CLOSED_FL", "sum")
main_table_agg = (
    main_table
    .groupby(group_key, dropna=False, as_index=False)
    .agg(LOAN_NUM_TOTAL=load_num_total_func, LOAN_NUM_CLOSED=load_num_closed_func)
)

LOGGER.info("Result table shape is %s.", main_table_agg.shape)
main_table_agg.to_csv(os.path.join(DATA_PATH, f"{RESULT_TABLE_NAME}.csv"), index=False)
main_table_agg.head(5)

INFO:__main__:Left join current table with 'CLIENT' table on - ['ID_CLIENT'].
INFO:__main__:Output shape is (15223, 16).
INFO:__main__:Left join current table with 'LOAN' table on - ['ID_CLIENT'].
INFO:__main__:Output shape is (21126, 17).
INFO:__main__:Left join current table with 'CLOSE_LOAN' table on - ['ID_LOAN'].
INFO:__main__:Output shape is (21126, 18).
INFO:__main__:Left join current table with 'JOB' table on - ['ID_CLIENT'].
INFO:__main__:Output shape is (21126, 22).
INFO:__main__:Left join current table with 'LAST_CREDIT' table on - ['ID_CLIENT'].
INFO:__main__:Output shape is (21126, 25).
INFO:__main__:Left join current table with 'SALARY' table on - ['ID_CLIENT'].
INFO:__main__:Output shape is (21126, 27).
INFO:__main__:Result table shape before drop duplicates and aggregation is (21126, 27).
INFO:__main__:Aggregate data with key ['CREDIT', 'GEN_TITLE', 'MARITAL_STATUS', 'AGE', 'SOCSTATUS_PENS_FL', 'REG_ADDRESS_PROVINCE', 'DEPENDANTS', 'OWN_AUTO', 'GEN_INDUSTRY', 'EDUCATION

,CREDIT,GEN_TITLE,MARITAL_STATUS,AGE,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,DEPENDANTS,OWN_AUTO,GEN_INDUSTRY,EDUCATION,SOCSTATUS_WORK_FL,JOB_DIR,TARGET,FL_PRESENCE_FL,TERM,AGREEMENT_RK,CHILD_TOTAL,POSTAL_ADDRESS_PROVINCE,PERSONAL_INCOME,GENDER,FST_PAYMENT,WORK_TIME,ID_CLIENT,FAMILY_INCOME,FACT_ADDRESS_PROVINCE,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
0,2000.0,БЕЗ РАБОТЫ,Вдовец/Вдова,56,1,Алтайский край,0,0,БЕЗ РАБОТЫ,Среднее специальное,0,БЕЗ РАБОТЫ,0,1,3,60469367,2,Алтайский край,6500.0,1,500.0,0.0,106805738,от 5000 до 10000 руб.,Алтайский край,1,1
1,2000.0,БЕЗ РАБОТЫ,Состою в браке,42,1,Ивановская область,1,0,БЕЗ РАБОТЫ,Среднее специальное,0,БЕЗ РАБОТЫ,0,0,6,61960111,1,Ивановская область,6000.0,0,845.0,0.0,106808401,от 5000 до 10000 руб.,Ивановская область,1,0
2,2000.0,Другое,Разведен(а),51,0,Курская область,0,0,Другие сферы,Среднее специальное,1,Участие в основ. деятельности,0,1,3,61102507,2,Курская область,5500.0,1,450.0,48.0,106806048,от 10000 до 20000 руб.,Курская область,1,1
3,2000.0,Индивидуальный предприниматель,Состою в браке,33,0,Томская область,3,0,Транспорт,Высшее,1,Участие в основ. деятельности,0,0,5,62494072,3,Томская область,15000.0,1,1165.0,4.0,106809909,от 20000 до 50000 руб.,Томская область,1,1
4,2000.0,Рабочий,Не состоял в браке,25,0,Орловская область,0,0,Здравоохранение,Высшее,1,Участие в основ. деятельности,0,1,3,66658224,0,Орловская область,6300.0,1,500.0,24.0,106814392,от 5000 до 10000 руб.,Орловская область,2,2


In [22]:
assert 0 == main_table.isna().sum().sum(), "Найдены пропущенные значения."
assert main_table.shape == main_table.drop_duplicates().shape, "Найдены дубли."

print("Всё ОК!")

Всё ОК!


### Попробуем посмотреть на простую модель

In [23]:
# классы несбалансированны
main_table_agg["TARGET"].value_counts(normalize=True)

TARGET
0    0.88097
1    0.11903
Name: proportion, dtype: float64

In [24]:
TRAIN_COLUMNS = list(filter(lambda column: "ID" not in column and "RK" not in column, main_table_agg))

TRAIN_DF = main_table_agg.loc[:, TRAIN_COLUMNS]
TRAIN_DF.dropna(subset=["TARGET"], inplace=True)
TRAIN_TARGET = TRAIN_DF["TARGET"]
TRAIN_DF.drop("TARGET", axis=1, inplace=True)

numeric_columns = TRAIN_DF.select_dtypes(np.number).columns
categorical_columns = TRAIN_DF.select_dtypes(object).columns

numeric_transformer = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
categorical_transformer = make_pipeline(SimpleImputer(fill_value="NONE", strategy="constant"), OneHotEncoder(handle_unknown="ignore"))

column_transformer = make_column_transformer(
    (numeric_transformer, list(numeric_columns)),
    (categorical_transformer, list(categorical_columns)),
)

pipeline = make_pipeline(column_transformer, LogisticRegression(max_iter=3_000))
pipeline.fit(TRAIN_DF, TRAIN_TARGET)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CREDIT', 'AGE',
                                                   'SOCSTATUS_PENS_FL',
                                                   'DEPENDANTS', 'OWN_AUTO',
                                                   'FL_PRESENCE_FL', 'TERM',
                                                   'CHILD_TOTAL',
                                                   'PERSONAL_INCOME', 'GENDER',
                                                   'FST_PAYMENT',
                                                   'LOAN_NUM_TOTAL',
                                                   'LOAN_...
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='NONE',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['GEN_TITLE',
                                                   'MARITAL_STATUS',
                                                   'REG_ADDRESS_PROVINCE',
                                                   'GEN_INDUSTRY', 'EDUCATION',
                                                   'JOB_DIR',
                                                   'POSTAL_ADDRESS_PROVINCE',
                                                   'FAMILY_INCOME',
                                                   'FACT_ADDRESS_PROVINCE'])])),
                ('logisticregression', LogisticRegression(max_iter=3000))])

In [25]:
for metric in "roc_auc", "accuracy", "f1", "precision", "recall":
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, TRAIN_DF, TRAIN_TARGET, cv=cv, verbose=False, scoring=metric)
    LOGGER.info("Mean '%s' score on CV: %s %s", metric, np.mean(scores), scores)

INFO:__main__:Mean 'roc_auc' score on CV: 0.6590344575743691 [0.65255969 0.66346652 0.66107716]


INFO:__main__:Mean 'accuracy' score on CV: 0.8793273541730177 [0.87901478 0.87977927 0.87918802]
INFO:__main__:Mean 'f1' score on CV: 0.013911338349540598 [0.00324675 0.0224359  0.01605136]
INFO:__main__:Mean 'precision' score on CV: 0.23216374269005846 [0.08333333 0.35       0.26315789]
INFO:__main__:Mean 'recall' score on CV: 0.007174392935982339 [0.00165563 0.0115894  0.00827815]


In [52]:
import joblib

pipeline.fit(TRAIN_DF, TRAIN_TARGET)
joblib.dump(pipeline, "../models/model.pkl")

['../models/model.pkl']

__Вывод:__ Возможно для простой модели и для самой задачи Uplift-Моделирования полученные результат считается приемлемым, но в целом выглядит не супер.